<a href="https://colab.research.google.com/github/wigenesfs/Imersao_IA_Alura_2024/blob/main/Imersao_IA_Alura_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sumarizador Imparcial de Discursos dos Deputados Federais**
###**Descrição**
Este projeto visa fornecer resumos concisos e informativos dos discursos proferidos por deputados federais na Câmara dos Deputados do Brasil, utilizando o poder dos Modelos de Linguagem de Grande Escala (LLMs), especificamente do Gemini do Google. Nosso foco é garantir a imparcialidade, eliminando vieses ideológicos ou políticos nos resumos gerados.

###**Funcionalidades Principais**
Coleta de Dados: Os discursos são coletados diretamente do site da Câmara dos Deputados, garantindo acesso à fonte primária da informação.
Resumos Imparciais: Utilizamos o LLM Gemini para gerar resumos concisos, livres de qualquer viés ideológico ou político. Nosso objetivo é apresentar o conteúdo de forma neutra e informativa.

###**Fonte de dados**
Banco de discursos dos Deputados Federais disponível em [Notas Taquigráficas da Câmara dos Deputados](https://www2.camara.leg.br/atividade-legislativa/discursos-e-notas-taquigraficas)

###**Como Usar**
Digitar o nome do parlamentar, podendo ser em maiúscula ou minúscula. É necessário também informar a data inicial e data final dos discurso proferidos.

Ex.: Para ver o resumo dos discursos proferidos pelo Deputado Tiririca entre 01/08/2017 e 04/09/2017, preencher os campos exibidos no modelo.

###**Contato**
Nome do desenvolvedor: Wigenes Sampaio
Email: [wigenesfs@gmail.com](wigenesfs@gmail.com)

###**Licença**
Este projeto está licenciado sob a licença [MIT](https://opensource.org/license/MIT).

# MODELO DE SUMARIZAÇÃO

#### Instalação das bibliotecas necessárias

In [1]:
!pip install -q -U google-generativeai

In [12]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from IPython.display import display, Markdown
import textwrap

#### Configurar a API key

In [3]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

#### Definir configurações para o modelo

In [4]:
gen_config = {
  "temperature": 0.2,
  "top_p": 0.95,
  "top_k": 2,
  "max_output_tokens": 8192,
}

safety_config = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

# Link base para o webscraping
LINK = "https://www.camara.leg.br/internet/sitaqweb/"

#### Listar modelos disponíveis

In [6]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


#### Iniciar o Modelo

In [7]:
#model = genai.GenerativeModel('gemini-pro')
model = genai.GenerativeModel("gemini-pro",
                              generation_config=gen_config,
                              safety_settings=safety_config)

#### Função para obter conteúdo dos discursos dos parlamentares

In [8]:
def fn_get_proxima_data(data):
    proximo_dia = pd.to_datetime(data) + pd.DateOffset(days=1)
    return proximo_dia


def fn_get_notas_taquigafricas(orador, data_inicial, data_final, link):
    discursos = []
    data_discurso = []
    df = pd.DataFrame()
    orador = orador.replace(" ", "+")
    link_base = link
    data_inicial = pd.to_datetime(data_inicial, dayfirst=True)
    data_final = pd.to_datetime(data_final, dayfirst=True)
    data_atual = pd.to_datetime(data_inicial, dayfirst=True)

    while data_atual <= data_final:
        inicio_dia = data_atual.day
        inicio_mes = data_atual.month
        inicio_ano = data_atual.year

        fim_dia = data_atual.day
        fim_mes = data_atual.month
        fim_ano = data_atual.year

        uri = f"https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txOrador={orador}&txPartido=&txUF=&dtInicio={inicio_dia}%2F{inicio_mes}%2F{inicio_ano}&dtFim={fim_dia}%2F{fim_mes}%2F{fim_ano}&txTexto=&txSumario=&basePesq=plenario&CampoOrdenacao=dtSessao&PageSize=50&TipoOrdenacao=ASC&btnPesq=Pesquisar"

        r = requests.get(uri)
        conteudo = r.content

        soup = BeautifulSoup(conteudo, 'html.parser')
        dados = soup.find('table', class_='variasColunas')

        if dados is not None:
            fn_busca_tabela(dados, link_base, discursos, data_discurso)

        data_atual = fn_get_proxima_data(data_atual)

        df = pd.DataFrame(list(zip(data_discurso, discursos)), columns=['data_discurso', 'discurso'])
    return df


def fn_busca_tabela(tabela, link_base, discursos, data_discurso):
    for linha in tabela.findAll('tr'):
        celula = linha.findAll('td')

        if len(celula) == 8:
            data_discurso.append((((celula[0].find(string=True)).replace('\r', '')).replace('\n', '')).replace('\t', ''))

            texto = fn_retorna_conteudo(
                link_base + (((celula[3].find('a', href=True)['href']).replace('\r', '')).replace('\n', '')).replace(
                    '\t', ''))
            discursos.append(texto)


def fn_retorna_conteudo(uri):
    retorna_resultado = requests.get(uri)
    retorna_conteudo = retorna_resultado.content
    dados = BeautifulSoup(retorna_conteudo, 'html.parser')
    dados = dados.find('p').getText()
    return(dados)

#### Função que resume o conteúdo dos discursos

In [9]:
def fn_explicar(dataframe, model):
    """
    Usa o modelo Gemini para explicar o conteúdo de cada linha de um dataframe.

    Args:
        dataframe: O dataframe a ser explicado.
        modeo: Modelo do Gemini a ser utilizado.

    Returns:
        Uma lista de strings, onde cada string é a explicação do Gemini para o conteúdo de uma linha do dataframe informado.
    """

    explicacoes = []
    for index, row in dataframe.iterrows():
        prompt = f"""
        Resuma o conteúdo da seguinte linha de um dataframe sem viés político ou ideológico. Se atenha ao que está no texto. O resumo deve conter no máximo 256 tokens:
        Em {row['data_discurso']}:
        {row.to_dict()}
        """
        resposta = model.generate_content(prompt)
        explicacoes.append(resposta.text)

    return explicacoes

In [26]:
# Função para markdown do resultado
def fn_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [28]:
chat = model.start_chat(history=[])
print("Bem-vindo ao Sumarizador Imparcial de Discursos dos Deputados Federais! Informe os dados ou digite 'fim' no primeiro campo para sair.\n\n")

def loop_de_analise_de_discursos(fn_get_notas_taquigafricas, fn_explicar, model, LINK):
  """
  Loop para analisar discursos de parlamentares usando as funções fornecidas.
  """
  nome_orador = ""

  while nome_orador.lower() != "fim":
    nome_orador = input("Digite o nome do Parlamentar (ou 'fim' para encerrar): ")
    if nome_orador.lower() == "fim":
      break

    data_inicio = input("Digite a data de início do discurso proferido (dd/mm/aaaa): ")
    data_fim = input("Digite a data final do discurso proferido (dd/mm/aaaa): ")

    df = fn_get_notas_taquigafricas(nome_orador, data_inicio, data_fim, LINK)

    if df is not None:  # Verifica se o dataframe foi criado com sucesso
      explicacoes = fn_explicar(df, model)
      for explicacao in explicacoes:
        display(fn_markdown(f'**Bot Sumarizador**: {explicacao}'))
        #print("Resumo: ", explicacao, "\n")
    else:
      print("Não foram encontradas notas taquigráficas para o período especificado.")


loop_de_analise_de_discursos(fn_get_notas_taquigafricas, fn_explicar, model, LINK)

Bem-vindo ao Sumarizador Imparcial de Discursos dos Deputados Federais! Informe os dados ou digite 'fim' no primeiro campo para sair.


Digite o nome do Parlamentar (ou 'fim' para encerrar): KIM KATAGUIRI
Digite a data de início do discurso proferido (dd/mm/aaaa): 06/02/2024
Digite a data final do discurso proferido (dd/mm/aaaa): 19/03/2024


> **Bot Sumarizador**: Em 6 de fevereiro de 2024, o deputado Kim Kataguiri (Bloco/UNIÃO-SP) discursou na Câmara dos Deputados para saudar o retorno do deputado Pauderney Avelino (DEM-AM) à Casa. Kataguiri elogiou a firmeza de Avelino na defesa de seus valores e expressou honra em tê-lo como colega de bancada.

> **Bot Sumarizador**: Em 27/02/2024, o deputado Kim Kataguiri criticou um projeto de lei que, segundo ele, inviabilizaria a correção da tabela do Imposto de Renda para isentar quem ganha até R$ 5.000,00, conforme prometido pelo presidente Lula durante a campanha eleitoral. Kataguiri argumentou que o projeto vincularia a isenção à faixa de até dois salários mínimos, o que impediria a correção da tabela pela inflação e tornaria a promessa de Lula inviável. Ele também criticou o projeto por impedir que futuros presidentes corrigissem a tabela pelo valor devido, configurando um "estelionato eleitoral".

> **Bot Sumarizador**: Em 19 de março de 2024, o deputado Kim Kataguiri criticou o governo Lula por propor medidas que afetam negativamente os motoristas e entregadores de aplicativo, como aumento de impostos e redução de rendimentos. Ele também expressou preocupação com a libertação de criminosos perigosos devido à Resolução do CNJ que encerra hospitais psiquiátricos de custódia. Kataguiri apelou à aprovação urgente de um projeto de lei que regulamenta esses hospitais e impede a libertação de 5.000 criminosos perigosos.

Digite o nome do Parlamentar (ou 'fim' para encerrar): FIM
